<a href="https://colab.research.google.com/github/ndow33/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS_122_Sampling_Confidence_Intervals_and_Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 122
## Sampling, Confidence Intervals, and Hypothesis Testing

## Recap: t-tests

In [0]:
# imports

In [0]:
url='https://raw.githubusercontent.com/ryanleeallred/datasets/master/adult.csv'

In [0]:
# Remove null values as these will throw off the ttest


In [0]:
# salary values


In [0]:
## what is the average age?


In [0]:
# restrict to high earners


In [0]:
# is age continuous?


In [0]:
# is the average age of high earners above the population average? yes.


In [0]:
# is there a sig difference between the age of high and low earners? yes.


In [0]:
# subset male and female low earners


# note the trailing space.

In [0]:
# is there a sig difference between the male and female low earners? yes.


## Prepare - examine other available hypothesis tests

If you had to pick a single hypothesis test in your toolbox, t-test would probably be the best choice - but the good news is you don't have to pick just one! Here's some of the others to be aware of:

In [0]:
# consider a crosstab. Are men and women equally distributed by salary?


In [0]:
# can we test that?


In [0]:
 # One-way chi square test

# Chi square can take any crosstab/table and test the independence of rows/cols
# The null hypothesis is that the rows/cols are independent -> low chi square
# The alternative is that there is a dependence -> high chi square
# Be aware! Chi square does *not* tell you direction/causation


In [0]:
# Is a distribution normal? This is a poisson distribution


In [0]:
# Is it normal?


In [0]:
# mean, std dev, number of obs



# Pretty clearly  normal

In [0]:
# display that!


In [0]:
# Poisson models arrival times and is related to the binomial (coinflip)


# Pretty clearly not normal

In [0]:
# display that!


And there's many more! `scipy.stats` is fairly comprehensive, though there are even more available if you delve into the extended world of statistics packages. As tests get increasingly obscure and specialized, the importance of knowing them by heart becomes small - but being able to look them up and figure them out when they *are* relevant is still important.

## Central Limit Theorem



In [0]:
# Flip a coin.


In [0]:
# Let me flip a coin twelve times.


In [0]:
# show that.


In [0]:
# what is the mean of this sample of coinflips?


In [0]:
# what if I do it again?


In [0]:
# and the mean?


In [0]:
# why are these means different? shouldn't it be the same at .5 for a fair coin?


**each of these 'samples' is an estimate of the distribution.** . 
https://www.youtube.com/watch?v=Kq7e6cj2nDw

In [0]:
# take 3000 samples of size 12, and make a list of their means.



In [0]:
# look at the first 10 means.


In [0]:
# make a df of one sample


In [0]:
# this df is clearly a binomial distribution (one and zero are the only options)


In [0]:
# now plot all 3000 sample means


In [0]:
# is this a normal distribution?


What does the Central Limit Theorem State? That no matter the initial distribution of the population, the distribution of sample means taken will approximate a normal distribution as $n \rightarrow \infty$.

This has very important implications for hypothesis testing and is precisely the reason why the t-distribution begins to approximate the normal distribution as our sample size increases. 

In [0]:
# let's pretend each sample included 100 coin flips, not just 12.

# the higher my sample size, the closer I get to .5!


In [0]:
# when I plot, it gets a little taller and a little narrower!


This outcome is what we call the Central Limit Theorem. No matter the distribution of my underlying data, the distribution of the estimates will be distributed normally.

In [0]:
# Compare sets of small and large samples


In [0]:
# Plot that for comparison: it gets taller, narrower, and more normal.


## Confidence Interval

![source](https://www.mathsisfun.com/data/images/ci95.gif)

With 95% confidence, the population mean is within these bounds. This is the [margin of error](https://www.monmouth.edu/polling-institute/reports/monmouthpoll_ia_111219/) we often seen in political polls.

Between the upper and lower bounds we can find 95% of the sample means.

<img src="https://github.com/ryanallredblog/ryanallredblog.github.io/blob/master/img/Confidence_Interval.png?raw=true" width=400>

s = standard deviation of the sample  
n = sample size  
t = test statistic  
X = sample mean
  
As sample size goes up, the standard error (i.e., the spread of our error) gets tighter.

In [0]:
# bring in the stats module (for the t-statistic)


In [0]:

    """
    Calculate a confidence interval around a sample mean for given data.
    Using t-distribution and two-tailed test, default 95% confidence. 

    Arguments:
    data - iterable (list or numpy array) of sample observations
    confidence - level of confidence for the interval

    Returns:
    tuple of (mean, lower bound, upper bound)
    """
    

In [0]:
# what's the conf interval for a sample of 500 coin flips?


# mean, upper bound, lower bound.

In [0]:
# try this again. It should capture the mean 95% of the time.


We want the t-statistic that corresponds to the probability cut off for a given t distribution  (a distribution with a certain 'degrees of freedom').  
`stats.t.ppf` is just looking up the corresponding t-statistic.

**Degrees of freedom**  
degrees of freedom equals the number of "observations" minus the number of required relations among the observations (e.g., the number of parameter estimates). For a 1-sample t-test, one degree of freedom is spent estimating the mean, and the remaining n - 1 degrees of freedom estimate variability.
https://blog.minitab.com/blog/statistics-and-quality-data-analysis/what-are-degrees-of-freedom-in-statistics

**Looking at stats.t.ppf**

In [0]:
# stats.t.ppf(# probability cutoff, # degrees of freedom)

# 95% confidence level -> .025

# 1 - confidence_level == .05 / 2 -> .025



## Graphically Represent a Confidence Interval

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# fake data - one samle of 42 coin flips

# kernel density estimate


In [0]:
# fake data - 50 samples of 12 coin flips


## Relationship between Confidence Intervals and T-tests

Confidence Interval == Bounds of statistical significance for our t-test

A sample mean that falls inside of our confidence interval will "FAIL TO REJECT" our null hypothesis

A sample mean that falls outside of our confidence interval will "REJECT" our null hypothesis

**Note that scipy has a built-in function to calculate the confidence interval**   
t.interval

**Rebuild the earlier function**
but using `t.interval` instead.

In [0]:
# Sample Size

# Degrees of Freedom

# The Mean of Means:

# Sample Standard Deviation

# Standard Error


In [0]:
'''You can roll your own CI calculation pretty easily. 
The only thing that's a little bit challenging 
is understanding the t stat lookup'''

# 95% confidence interval

A null hypothesis that's just inside of our confidence interval == fail to reject



In [0]:
# pick a value for my null hypothesis and check the p-value


In [0]:
# what if I pick a null hypothesis that's just outside of our confidence interval? 


In [0]:
# I would only see this 5% of the time, so it's unlikely and I reject the null hypothesis.
# A comparison of what we EXPECT and what we OBSERVE.

## Chi Squared Test $\chi^{2}$

A test for dependence of categorical variables. (ie, are they correlated?)

In [0]:
# read back in the earlier data.


In [0]:
# just the numeric columns


In [0]:
# just the non-numeric columns (ie, categorical)


In [0]:
# create some binned categories 


In [0]:
# sex


In [0]:
# binned category


In [0]:
# create a set of female counts


In [0]:
# create a set of male counts


In [0]:
#Plots the bar chart


## Expected Value Calculation
\begin{align}
expected_{i,j} =\frac{(row_{i} \text{total})(column_{j} \text{total}) }{(\text{total observations})}  
\end{align}

## Chi-Squared Statistic with Numpy

\begin{align}
\chi^2 = \sum \frac{(observed_{i}-expected_{i})^2}{(expected_{i})}
\end{align}

For the $observed$ values we will just use a version of our contingency table without the margins as a numpy array. In this way, if our observed values array and our expected values array are the same shape, then we can subtract them and divide them directly which makes the calculations a lot cleaner. No for loops!

In [0]:
# Array broadcasting will work with numpy arrays but not python lists


In [0]:
# Degrees of Freedom of a Chi-squared test

#degrees_of_freedom = (num_rows - 1)(num_columns - 1)


# Calculate Degrees of Freedom


## Run a $\chi^{2}$ Test using Scipy

Null Hypothesis: Hours worked per week bins is **independent** of sex. 

Due to a p-value of 0, we REJECT the null hypothesis that hours worked per week and sex are independent, and conclude that there is an association between hours worked per week and sex.

### Another example

In [0]:
# sex and salary

In [0]:
# chi square test

In [0]:
# age and salary

In [0]:
# crosstab

In [0]:
# chi square test